In [1]:
# Load packages
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Load dataset (CIFAR10)
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#  Loaders for training and testing set
batch_size = 64 

trainloader = torch.utils.data.DataLoader(training_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
testloader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size)

In [4]:
# Figure out whether to use cpu, mps or cuda.

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')
print(f"Using {device} device")

Using mps device


In [ ]:
# Create the NN
class WideResnet(nn.Module):
    def __init__(self):
        super().__init__()
        

In [21]:
from datetime import datetime
datetime.now().strftime("%m-%d-H%H")

'05-07-H13'

In [12]:
torch.log1p(torch.exp(torch.tensor(-5)))

tensor(0.0067)

In [77]:
from torch.distributions import Normal, kl_divergence
import torch


# Example values
out_features = 10
in_features = 5

# Initialize parameters
u = torch.Tensor(out_features, 1).uniform_(-0.1, 0.1)
v = torch.Tensor(1, in_features).uniform_(-0.1, 0.1)
u_rho = torch.Tensor(out_features, 1).uniform_(-0.1, 0.1)
v_rho = torch.Tensor(1, in_features).uniform_(-0.1, 0.1)

# Compute standard deviations using Softplus
u_sigma = torch.log1p(torch.exp(u_rho))
v_sigma = torch.log1p(torch.exp(v_rho))

# Define the posterior distributions
u_posterior = Normal(u, u_sigma)
v_posterior = Normal(v, v_sigma)

# Define the prior distribution
weight_prior = Normal(0, 1)

# Compute the KL divergences
kl_u = kl_divergence(u_posterior, weight_prior).sum()
kl_v = kl_divergence(v_posterior, weight_prior).sum()

print(f"KL divergence for u: {kl_u.item()}")
print(f"KL divergence for v: {kl_v.item()}")

KL divergence for u: 1.1262069940567017
KL divergence for v: 0.6226749420166016


In [78]:
u_sample = Normal(u, u_sigma).rsample()
v_sample = Normal(v, v_sigma).rsample()
print(u_sample)
print(v_sample)

tensor([[-0.3965],
        [-0.1859],
        [ 1.3076],
        [-0.1007],
        [ 0.4319],
        [ 0.1714],
        [-0.2168],
        [-0.1110],
        [ 0.1618],
        [ 0.3294]])
tensor([[-0.8564, -0.7156, -0.3526, -0.9760,  0.2742]])


In [89]:
rs = torch.mm(u_sample, v_sample)
rs

tensor([[ 0.3395,  0.2837,  0.1398,  0.3870, -0.1087],
        [ 0.1592,  0.1330,  0.0655,  0.1814, -0.0510],
        [-1.1198, -0.9357, -0.4611, -1.2762,  0.3586],
        [ 0.0862,  0.0721,  0.0355,  0.0983, -0.0276],
        [-0.3699, -0.3091, -0.1523, -0.4215,  0.1184],
        [-0.1468, -0.1226, -0.0604, -0.1673,  0.0470],
        [ 0.1856,  0.1551,  0.0764,  0.2116, -0.0594],
        [ 0.0951,  0.0794,  0.0391,  0.1083, -0.0304],
        [-0.1386, -0.1158, -0.0571, -0.1579,  0.0444],
        [-0.2821, -0.2357, -0.1162, -0.3215,  0.0903]])

In [104]:
w = torch.full((10, 5), 10)
w * rs

tensor([[  3.3955,   2.8372,   1.3981,   3.8696,  -1.0873],
        [  1.5916,   1.3299,   0.6553,   1.8139,  -0.5097],
        [-11.1981,  -9.3568,  -4.6109, -12.7620,   3.5860],
        [  0.8625,   0.7207,   0.3551,   0.9829,  -0.2762],
        [ -3.6988,  -3.0907,  -1.5230,  -4.2154,   1.1845],
        [ -1.4677,  -1.2264,  -0.6043,  -1.6727,   0.4700],
        [  1.8563,   1.5510,   0.7643,   2.1155,  -0.5944],
        [  0.9506,   0.7943,   0.3914,   1.0833,  -0.3044],
        [ -1.3858,  -1.1579,  -0.5706,  -1.5793,   0.4438],
        [ -2.8212,  -2.3573,  -1.1616,  -3.2152,   0.9034]])

In [108]:
(w * u_sample) * v_sample

tensor([[  3.3955,   2.8372,   1.3981,   3.8696,  -1.0873],
        [  1.5916,   1.3299,   0.6553,   1.8139,  -0.5097],
        [-11.1981,  -9.3568,  -4.6109, -12.7620,   3.5860],
        [  0.8625,   0.7207,   0.3551,   0.9829,  -0.2762],
        [ -3.6988,  -3.0907,  -1.5230,  -4.2154,   1.1845],
        [ -1.4677,  -1.2264,  -0.6043,  -1.6727,   0.4700],
        [  1.8563,   1.5510,   0.7643,   2.1155,  -0.5944],
        [  0.9506,   0.7943,   0.3914,   1.0833,  -0.3044],
        [ -1.3858,  -1.1579,  -0.5706,  -1.5793,   0.4438],
        [ -2.8212,  -2.3573,  -1.1616,  -3.2152,   0.9034]])

In [117]:
(w * v_sample) * u_sample

tensor([[  3.3955,   2.8372,   1.3981,   3.8696,  -1.0873],
        [  1.5916,   1.3299,   0.6553,   1.8139,  -0.5097],
        [-11.1981,  -9.3568,  -4.6109, -12.7620,   3.5860],
        [  0.8625,   0.7207,   0.3551,   0.9829,  -0.2762],
        [ -3.6988,  -3.0907,  -1.5230,  -4.2154,   1.1845],
        [ -1.4677,  -1.2264,  -0.6043,  -1.6727,   0.4700],
        [  1.8563,   1.5510,   0.7643,   2.1155,  -0.5944],
        [  0.9506,   0.7943,   0.3914,   1.0833,  -0.3044],
        [ -1.3858,  -1.1579,  -0.5706,  -1.5793,   0.4438],
        [ -2.8212,  -2.3573,  -1.1616,  -3.2152,   0.9034]])

In [143]:
u = torch.nn.Parameter(torch.Tensor(10, 1).uniform_(-0.1, 0.1))
v = torch.nn.Parameter(torch.Tensor(1, 5, 3, 3).uniform_(-0.1, 0.1))

v = v.view(1, 5, 3, 3)

u*v

RuntimeError: The size of tensor a (10) must match the size of tensor b (3) at non-singleton dimension 2

In [147]:
string = "hejldjasweight"

if "weight" in string:
    print("This works")

This works


In [11]:
import numpy as np
np.log(np.expm1(np.sqrt(0.001 / (1. - 0.001))))

-3.437516381367939

In [7]:
import torch
torch.log1p(torch.exp(torch.tensor(-1)))

tensor(0.3133)

In [26]:
(19 + 1) % 20 == 0

True